In [ ]:
import sys
sys.path.insert(0,'../')
from util.train_128gray_resnet import resnet128
import torch
imagnet_classifier = resnet128.resnet50().to(0)
checkpoint_path = '/private/home/zizhao/work/checkpoint_fmri/resnset128_scratch/model_best.pth.tar'
checkpoint = torch.load(checkpoint_path)
print('checkpoint prec', checkpoint['best_prec1'])
imagnet_classifier = torch.nn.DataParallel(imagnet_classifier, device_ids=[0]).cuda()
imagnet_classifier.load_state_dict(checkpoint['state_dict'])
imagnet_classifier.eval()
classifier_mean = torch.cuda.FloatTensor([0.43, 0.43, 0.43]).view(1,3,1,1)
classifier_std = torch.cuda.FloatTensor([0.23, 0.23, 0.23]).view(1,3,1,1)

In [ ]:
import torchvision.datasets as datasets
import os
from torchvision import transforms

valdir = os.path.join('/datasets01/imagenet_resized_144px/060718/061417', 'val')
test_data_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(valdir, transforms.Compose([
        transforms.Grayscale(num_output_channels=1), # zz
        transforms.Resize(144),
        # transforms.Resize(224),
        transforms.CenterCrop(128),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])),
    batch_size=64, shuffle=False,
    num_workers=4, pin_memory=True)

In [ ]:
def merge_kpsace(img, mask):
    
    p_fft = rfft(img[:,:1,:,:])
    res = p_fft * mask 
    res = ifft(res)
    return res[:,:1,:,:].repeat(1,3,1,1)

from test_kspace_recom_classification import accuracy
import numpy as np
from models.fft_utils import create_mask
from models.fft_utils import RFFT, IFFT
import torchvision.utils as tvutil
import matplotlib.pyplot as plt
%matplotlib inline
rfft = RFFT()
ifft = IFFT()
for ratio in np.arange(0.1,1,0.1):
    mask = create_mask(mask_fraction=ratio).cuda()
    accs = []
    for i, (imgs, labels) in enumerate(test_data_loader):
        imgs = imgs[:,:1,:,:].cuda()
        imgs = imgs.add(1).div(2)
        
#         print(img.shape)
#         import pdb; pdb.set_trace()
        imgs = imgs.repeat(1,3,1,1)
        inputs = (imgs - classifier_mean) / classifier_std
        inputs = merge_kpsace(inputs, mask)
#         plt.figure(figsize=(10,10))
#         plt.imshow(tvutil.make_grid(imgs[:4], normalize=True, scale_each=True).permute(1,2,0))
        acc = accuracy(imagnet_classifier(inputs).cpu(), labels, topk=(1,5))
        acc = [a.item() for a in acc]
        accs.append(acc)
        if i > 1: break

    print('ratio {} acc {}'.format(mask.mean(),  np.mean(np.array(accs), 0)))

In [ ]:
64*1000